# DCGAN
> Defines the DCGAN architecture

In [ ]:
#hide
IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
  !pip3 install -Uqq fastbook

     |████████████████████████████████| 727kB 10.3MB/s 
     |████████████████████████████████| 1.0MB 41.9MB/s 
     |████████████████████████████████| 194kB 50.1MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 40kB 5.3MB/s 
     |████████████████████████████████| 92kB 11.0MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 2.6MB 49.2MB/s 


In [ ]:
#hide
if IN_COLAB:
  from pathlib import Path
  from nbdev.imports import Config
  project_path = Path('/content/drive/My Drive/Colab Notebooks/github/dcgan')
  get_ipython().magic(f'cd {project_path}')
  get_ipython().magic(f'cd {Config().nbs_path}')

In [ ]:
# default_exp models

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from fastai.vision.all import *

## Generator

In [ ]:
#export
def build_conv_layer(ch_in:int,
                     ch_out:int,
                     ks:int,
                     stride:int,
                     padding:int=0,
                     bias:bool=True,
                     transpose:bool=False,
                     mean_weight:float=0.0,
                     std_weight:float=0.02
                     ):
  if transpose:
    conv = nn.ConvTranspose2d(ch_in,
                              ch_out,
                              ks,
                              stride,
                              padding,
                              bias=bias
                              )
  else:
    conv = nn.Conv2d(ch_in,
                     ch_out,
                     ks,
                     stride,
                     padding,
                     bias=bias
                     )
    
  nn.init.normal_(conv.weight, mean_weight, std_weight)
  return conv

In [ ]:
#export
def build_bn(ch_in:int, 
             mean_weight:float=0.0,
             std_weight:float=0.02,
             bias_const:float=0.0
             ):
  bn = nn.BatchNorm2d(ch_in)
  nn.init.normal_(bn.weight, mean_weight, std_weight)
  nn.init.constant_(bn.bias, bias_const)
  return bn

In [ ]:
#export
def dcgan_generator(z_dim:int,
                    ch_in:int,
                    hidden_dim:int
                    ):
  layers = []
  layers += build_mnist_gen_arch(z_dim, ch_in, hidden_dim)

  return nn.Sequential(*layers)

def build_mnist_gen_arch(z_dim:int,
                         ch_in:int,
                         hidden_dim:int
                         ):
  
  layers = [build_conv_layer(z_dim, hidden_dim * 4, ks=3, stride=2, bias=False, transpose=True),
            build_bn(hidden_dim * 4),
            nn.ReLU(),
            build_conv_layer(hidden_dim * 4, hidden_dim * 2, ks=4, stride=1, bias=False, transpose=True),
            build_bn(hidden_dim * 2),
            nn.ReLU(),
            build_conv_layer(hidden_dim * 2, hidden_dim,  ks=3, stride=2, bias=False, transpose=True),
            build_bn(hidden_dim),
            nn.ReLU(),
            build_conv_layer(hidden_dim, ch_in, ks=4, stride=2, bias=False, transpose=True),
            nn.Tanh()]

  return layers

### Tests

In [ ]:
noise = torch.randn(1, 64, 1, 1)
m     = dcgan_generator(z_dim=64, 
                        ch_in=1, 
                        hidden_dim=64)

with torch.no_grad():
  out = m(noise)

test_eq(out.shape, (1, 1, 28, 28))

## Discriminator

In [ ]:
#export
def dcgan_discriminator(ch_in:int,
                        hidden_dim:int
                        ):
  layers = []
  layers += build_mnist_disc_arch(ch_in, hidden_dim)

  return nn.Sequential(*layers)

def build_mnist_disc_arch(ch_in:int,
                          hidden_dim:int
                          ):
  
  out_units = 1 # since discriminator has to estimate real/fake (binary) probability
  layers = [build_conv_layer(ch_in, hidden_dim, ks=4, stride=2, bias=False),
            build_bn(hidden_dim),
            nn.LeakyReLU(negative_slope=0.2),
            build_conv_layer(hidden_dim, hidden_dim * 2, ks=4, stride=2, bias=False),
            build_bn(hidden_dim * 2),
            nn.LeakyReLU(negative_slope=0.2),
            build_conv_layer(hidden_dim * 2, 1, ks=4, stride=2, bias=False)]
  return layers

### Tests

In [ ]:
img = torch.randn(1, 1, 28, 28)
m   = dcgan_discriminator(ch_in=1, hidden_dim=16)

with torch.no_grad():
  out = m(img)

test_eq(out.shape, (1, 1, 1, 1))

## Full Model

In [ ]:
#export
class DCGAN(nn.Module):
  def __init__(self, 
               ch_in:int, 
               z_dim:int, 
               gen_hidden_dim:int=64, 
               disc_hidden_dim:int=16):
    
    super().__init__()

    self.D = dcgan_discriminator(ch_in=ch_in, 
                                 hidden_dim=disc_hidden_dim)
    self.G = dcgan_generator(z_dim=z_dim, 
                             ch_in=ch_in, 
                             hidden_dim=gen_hidden_dim)

  def forward(self, noise, real_image):
    fake_image = self.G(noise)

    return fake_image

### Tests

In [ ]:
dcgan_model = DCGAN(ch_in=1, z_dim=64) # z_dim: dimension of the random noise vector

img1 = torch.randn(4,64,1,1)
img2 = torch.randn(4,1,28,28)

In [ ]:
%%time
with torch.no_grad(): dcgan_output = dcgan_model(img1, img2)

CPU times: user 11.8 ms, sys: 974 µs, total: 12.8 ms
Wall time: 13.9 ms


In [ ]:
test_eq(len(dcgan_output), 4)
test_eq(dcgan_output.shape, img2.shape)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_data.ipynb.
Converted 01_models.ipynb.
Converted index.ipynb.
